# Advance Regression Challenge

Now that you have two new regression methods at your fingertips, it's time to give them a spin. In fact, for this challenge, let's put them together! Pick a dataset of your choice with a binary outcome and the potential for at least 15 features. If you're drawing a blank, the crime rates in 2013 dataset has a lot of variables that could be made into a modelable binary outcome.

Engineer your features, then create three models. Each model will be run on a training set and a test-set (or multiple test-sets, if you take a folds approach). The models should be:

- Vanilla logistic regression
- Ridge logistic regression
- Lasso logistic regression

If you're stuck on how to begin combining your two new modeling skills, here's a hint: the [SKlearn LogisticRegression method](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) has a "penalty" argument that takes either 'l1' or 'l2' as a value.

In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you think that particular model is the best of the three. Also reflect on the strengths and limitations of regression as a modeling approach. Were there things you couldn't do but you wish you could have done?

Record your work and reflections in a notebook to discuss with your mentor.

In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import sklearn
from sklearn import linear_model
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format
%matplotlib inline
sns.set_style('white')
# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
from sklearn import linear_model
from sklearn import preprocessing
import math
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

## Load/Clean Data

For this challenge the Crime Data for California, from the Linear Regression challenge, will be used:

[Download the Excel file here](https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/table-8/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls) on crime data in New York State in 2013, provided by the FBI: UCR.

Prepare this data to model with multivariable regression (including data cleaning if necessary) according to this specification:

Propertycrime=α+Population+Population^2+Murder+Robbery
 
The 'population' variable is already set for you, but you will need to create the last three features. Robbery and Murder are currently continuous variables. For this model, please use these variables to create categorical features where values greater than 0 are coded 1, and values equal to 0 are coded 0. You'll use this data and model in a later assignment- for now, just write the code you need to get the data ready. Don't forget basic data cleaning procedures, either! Do some graphing to see if there are any anomalous cases, and decide how you want to deal with them.

1) The figures shown in this column for the offense of rape were reported using the revised Uniform Crime Reporting (UCR) definition of rape. See Data Declaration for further explanation.

2) The figures shown in this column for the offense of rape were reported using the legacy UCR definition of rape. See Data Declaration for further explanation.

3) The FBI does not publish arson data unless it receives data from either the agency or the state for all 12 months of the calendar year.

**Link to the [Data Declaration](https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/table-8/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls/@@template-layout-view?override-view=data-declaration) page**

In [2]:
# import CA data and show format
df = pd.read_csv('table_8_offenses_known_to_law_enforcement_california_by_city_2013.csv')
# rename the columns
df.rename(columns={'Table 8': 'City', 'Unnamed: 1': 'Population', 'Unnamed: 2': 'Violent_crime', 
                   'Unnamed: 3': 'Murder', 'Unnamed: 4': 'Rape (revised definition)', 
                   'Unnamed: 5': 'Rape', 'Unnamed: 6': 'Robbery', 
                   'Unnamed: 7': 'Aggravated_assault', 'Unnamed: 8': 'Property_crime', 
                   'Unnamed: 9': 'Burglary', 'Unnamed: 10': 'Larceny_theft', 'Unnamed: 11': 'Motor_vehicle_theft',
                   'Unnamed: 12': 'Arson'}, inplace=True)
# drop title rows from the csv file
df=df.drop(['Unnamed: 13', 'Rape (revised definition)'], axis=1)
df=df.drop([0, 1, 2, 3, 466, 467], axis=0)
df.head()

,City,Population,Violent_crime,Murder,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny_theft,Motor_vehicle_theft,Arson
4,Adelanto,"31,165",198,2,15,52,129,886,381,372,133,17
5,Agoura Hills,"20,762",19,0,2,10,7,306,109,185,12,7
6,Alameda,"76,206",158,0,10,85,63,"1,902",287,"1,285",330,17
7,Albany,"19,104",29,0,1,24,4,557,94,388,75,7
8,Alhambra,"84,710",163,1,9,81,72,"1,774",344,"1,196",234,7


In [3]:
# add state column
df['State']='CA'
# convert all numerical columns to integers
for x in df.index:
    df['Population'].loc[x]=int(df['Population'].loc[x].replace(',', ''))
    df['Violent_crime'].loc[x]=int(df['Violent_crime'].loc[x].replace(',', ''))
    df['Murder'].loc[x]=int(df['Murder'].loc[x].replace(',', ''))
    df['Rape'].loc[x]=int(df['Rape'].loc[x].replace(',', ''))
    df['Robbery'].loc[x]=int(df['Robbery'].loc[x].replace(',', ''))
    df['Aggravated_assault'].loc[x]=int(df['Aggravated_assault'].loc[x].replace(',', ''))
    df['Property_crime'].loc[x]=int(df['Property_crime'].loc[x].replace(',', ''))
    df['Burglary'].loc[x]=int(df['Burglary'].loc[x].replace(',', ''))
    df['Larceny_theft'].loc[x]=int(df['Larceny_theft'].loc[x].replace(',', ''))
    df['Motor_vehicle_theft'].loc[x]=int(df['Motor_vehicle_theft'].loc[x].replace(',', ''))
    df['Arson'].loc[x]=int(df['Arson'].loc[x].replace(',', ''))

/Users/kyleknoebel/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
# add a population^2 column and add inputs
df['Population^2']=df['Population']**2

# add murder cat column and add inputs
df['Murder Cat']=''
for x in df.index:
    if df['Murder'].loc[x] == 0:
          df['Murder Cat'].loc[x]=df['Murder Cat'].loc[x].replace('', '0')
    else:
          df['Murder Cat'].loc[x]=df['Murder Cat'].loc[x].replace('', '1')
# add robbery cat column and add inputs
df['Robbery Cat']=''
for x in df.index:
    if df['Robbery'].loc[x] == 0:
          df['Robbery Cat'].loc[x]=df['Robbery Cat'].loc[x].replace('', '0')
    else:
          df['Robbery Cat'].loc[x]=df['Robbery Cat'].loc[x].replace('', '1')
# convert Robbery Cat and Murder Cat to integers
for x in df.index:
    df['Murder Cat'].loc[x]=int(df['Murder Cat'].loc[x])
    df['Robbery Cat'].loc[x]=int(df['Robbery Cat'].loc[x])         
df.head()

/Users/kyleknoebel/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,City,Population,Violent_crime,Murder,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny_theft,Motor_vehicle_theft,Arson,State,Population^2,Murder Cat,Robbery Cat
4,Adelanto,31165,198,2,15,52,129,886,381,372,133,17,CA,971257225,1,1
5,Agoura Hills,20762,19,0,2,10,7,306,109,185,12,7,CA,431060644,0,1
6,Alameda,76206,158,0,10,85,63,1902,287,1285,330,17,CA,5807354436,0,1
7,Albany,19104,29,0,1,24,4,557,94,388,75,7,CA,364962816,0,1
8,Alhambra,84710,163,1,9,81,72,1774,344,1196,234,7,CA,7175784100,1,1


For the for the Logistic regression in Stasmodel, a 'Robbery Prob' column will be added to the dataset as the probability of getting robbed depending on population. First we will find the max population in the dataframe:  

In [5]:
# find the max population
df['Population'].max()

3878725

In order to make this a probability, all population values will be divided by 10000000.

In [6]:
df['Robbery Prob']=df['Population']/10000000
df.head()

,City,Population,Violent_crime,Murder,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny_theft,Motor_vehicle_theft,Arson,State,Population^2,Murder Cat,Robbery Cat,Robbery Prob
4,Adelanto,31165,198,2,15,52,129,886,381,372,133,17,CA,971257225,1,1,0.003
5,Agoura Hills,20762,19,0,2,10,7,306,109,185,12,7,CA,431060644,0,1,0.002
6,Alameda,76206,158,0,10,85,63,1902,287,1285,330,17,CA,5807354436,0,1,0.008
7,Albany,19104,29,0,1,24,4,557,94,388,75,7,CA,364962816,0,1,0.002
8,Alhambra,84710,163,1,9,81,72,1774,344,1196,234,7,CA,7175784100,1,1,0.008


In [7]:
df['Robbery Cat'].value_counts()

1    427
0     35
Name: Robbery Cat, dtype: int64

Population, population squared, murder and robbery will be the continuous features. These features were chosen becasue they match the linear regression model, which will hopefully make all the models easier to compare afterwards.  

## 1) Vanilla logistic regression

### Statsmodel

In [9]:
# Declare predictors.
X_statsmod = df[['Murder', 'Population', 'Population^2', 'Robbery']]

# The Statsmodels formulation requires a column with constant value 1 that
# will act as the intercept.
X_statsmod['intercept'] = 1 

# Declare and fit the model.
logit = sm.Logit(df['Robbery Prob'], X_statsmod)
result = logit.fit()

# Lots of information about the model and its coefficients, but the
# accuracy rate for predictions is missing.
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.008413
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:           Robbery Prob   No. Observations:                  462
Model:                          Logit   Df Residuals:                      457
Method:                           MLE   Df Model:                            4
Date:                Sun, 30 Dec 2018   Pseudo R-squ.:                     inf
Time:                        10:23:53   Log-Likelihood:                -3.8868
converged:                       True   LL-Null:                        0.0000
                                        LLR p-value:                     1.000
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Murder           0.0292      0.081      0.359      0.720      -0.130       0.188
Population    4.057e-06

/Users/kyleknoebel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kyleknoebel/anaconda3/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:3313: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.llf/self.llnull


This model is very inaccurate, most likely becasue the Robbery Prob feature is not a good indicator of probability of robbery. However, it was a good feature to use to practice with the Statsmodel logistic regression tool. 

### sklearn

In [10]:
# Declare a logistic regression classifier.
# Parameter regularization coefficient C described above.
lr = LogisticRegression(C=1e9)
y = df['Robbery Cat']
X = df[['Murder', 'Population', 'Population^2', 'Robbery']]

# Fit the model.
fit = lr.fit(X, y)

# Display.
print('Coefficients')
print(fit.coef_)
print(fit.intercept_)
pred_y_sklearn = lr.predict(X)

print('\n Accuracy from model')
print(pd.crosstab(pred_y_sklearn, y))

print('\n Percentage accuracy')
print(lr.score(X, y))

Coefficients
[[7.41821767e-17 1.26802203e-12 1.45745763e-08 1.90136033e-15]]
[1.35061799e-16]

 Accuracy from model
Robbery Cat   0    1
row_0               
1            35  427

 Percentage accuracy
0.9242424242424242


This model is much more accurate, most likely as it relies on a binary outcome of robbery. 

## Ridge logistic regression

In [11]:
# create df_run dataset to only include features discussed above
df_run=df[['Murder', 'Population', 'Population^2', 'Robbery', 'Robbery Cat']]

In [23]:
# Define the training and test sizes.
Y = df['Robbery Cat']
trainsize = int(df.shape[0] / 2)
df_test = df_run.iloc[trainsize:, :].copy()
df_train = df_run.iloc[:trainsize, :].copy()

# Set up the regression model to predict defaults using all other
# variables as features.
ridgeregr = LogisticRegression(penalty='l2', random_state=0, fit_intercept=False, solver='lbfgs', 
                               multi_class='multinomial')
Y_train = df_train['Robbery Cat'].values.reshape(-1, 1)
X_train = df_train.loc[:, ~(df_train.columns).isin(['Robbery Cat'])]
ridgeregr.fit(X_train, Y_train)
print('\nR-squared simple model:')
print(ridgeregr.score(X_train, Y_train))


R-squared simple model:
0.935064935064935


/Users/kyleknoebel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
# Re-run the model with the new features.
X_train2 = df_train.loc[:, ~(df_train.columns).isin(['Robbery Cat'])]
ridgeregr.fit(X_train2, Y_train)
print('\nR-squared complex model:')
print(ridgeregr.score(X_train2, Y_train))


R-squared complex model:
0.935064935064935


/Users/kyleknoebel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The model is just as accurate on small and large training sizes. Since ridge regression eliminates overfitting, this could be a sign that there is not much multicollinearity in the data.  

## Lasso logistic regression

In [31]:
# Define the training and test sizes.
trainsize = int(df.shape[0] / 2)
df_test = df_run.iloc[trainsize:, :].copy()
df_train = df_run.iloc[:trainsize, :].copy()

Y_train = df_train['Robbery Cat'].values.reshape(-1, 1)
X_train = df_train.loc[:, ~(df_train.columns).isin(['Robbery Cat'])]
# Test the simpler model with smaller coefficients.
Y_test = df_test['Robbery Cat'].values.reshape(-1, 1)
X_test = df_test.loc[:, ~(df_test.columns).isin(['Robbery Cat'])]

# Small number of parameters.
lassregr = LogisticRegression(penalty='l1', random_state=0, fit_intercept=False, solver='liblinear')
lassregr.fit(X_train, Y_train)
print('R² for the cimpler model:')
print(lassregr.score(X_train, Y_train))

R² for the model with few features:
0.935064935064935


/Users/kyleknoebel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
X_train2 = df_train.loc[:, ~(df_train.columns).isin(['Robbery Cat'])]

# Re-run the model with the new features.
X_train2 = df_train.loc[:, ~(df_train.columns).isin(['Robbery Cat'])]
lassregr.fit(X_train2, Y_train)
print('\nR-squared complex model:')
print(lassregr.score(X_train2, Y_train))


R-squared complex model:
0.935064935064935


/Users/kyleknoebel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


This accuracy of this model also did not change between training and test populations. As Lasso logistic regression eliminates some parameters, it is possible that none of the features zeroed out, or the ones that did had no effect on the outcome of the model. 